In [4]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests

#URLs for each exchange's API - Only these two out of the given APIs are working
exchange_urls = {
    "Wazirx": "https://api.wazirx.com/api/v2/tickers",
    "CoinDCX": "https://api.coindcx.com/exchange/ticker",
}


column_names = ["Exchange", "Base Unit", "Quote Unit", "Low", "High", "Last", "Volume", "Sell", "Buy", "At", "Name"]


def fetch_exchange_data(exchange_name, url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        exchange_data = response.json()
        return exchange_data
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from {exchange_name}: {e}")
        return None


scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pawan/Downloads/creds.json', scope)
gc = gspread.authorize(credentials)

sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1k9vZJsj8vSzxu7r7rQyrcBj-2Cnw5lTNPVgbY59Sr3I/edit?pli=1#gid=0').sheet1


def main():
    try:
        while True:
            sheet.clear()
            
            sheet.append_row(column_names)

            row_count = 0

            for exchange_name, exchange_url in exchange_urls.items():
                exchange_data = fetch_exchange_data(exchange_name, exchange_url)
                if exchange_data:
                    top_pairs = sorted(exchange_data.values(), key=lambda x: float(x.get('volume', 0)), reverse=True)[:20]
                    
                    for pair_data in top_pairs:
                        row_data = [exchange_name, pair_data.get('base_unit', ''), pair_data.get('quote_unit', ''),
                                    pair_data.get('low', ''), pair_data.get('high', ''), pair_data.get('last', ''),
                                    pair_data.get('volume', ''), pair_data.get('sell', ''), pair_data.get('buy', ''),
                                    pair_data.get('at', ''), pair_data.get('name', '')]
                        sheet.append_row(row_data)
                        row_count += 1
                        
                        #Getting Top 20 updated data
                        if row_count >= 20:
                            break
                    if row_count >= 20:
                        break
            time.sleep(60)
    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")

if __name__ == "__main__":
    main()


Process interrupted. Exiting...
